In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('./retail_train.csv')
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [4]:
popularity1 = data.groupby('item_id')['sales_value'].sum().reset_index()

Любое значение log(sales_values < 1) = отрицательному числу, а занчит отрицательной вероятности, следовательно заменяем их на 1, что даст вероятность 0.

In [5]:
popularity1['sales_value'].where(popularity1['sales_value'] > 1, 1, inplace=True)
log_sales_value = np.log(popularity1['sales_value'])
popularity1['log_weight'] = log_sales_value / log_sales_value.sum()

In [6]:
popularity1.head()

,item_id,sales_value,log_weight
0,25671,20.94,0.000013
1,26081,1.00,0.000000
2,26093,1.59,0.000002
3,26190,1.54,0.000002
4,26355,1.98,0.000003


In [7]:
popularity1['log_weight'].sum()

0.9999999999999999

In [8]:
popularity1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89051 entries, 0 to 89050
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   item_id      89051 non-null  int64  
 1   sales_value  89051 non-null  float64
 2   log_weight   89051 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 2.0 MB


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [9]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    items = np.array(items_weights.iloc[:,0])
    p = np.array(items_weights.iloc[:,1])
    recs = np.random.choice(items, p=p, size=n, replace=False)
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    return recs.tolist()

In [10]:
%%time
items_weights = popularity1[['item_id', 'log_weight']]
weighted_random_recommendation(items_weights, n=5)

Wall time: 7.98 ms


[1024306, 857620, 6533497, 6514320, 917525]

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

Из файла списки обратно возвращаются в виде строк, это функция возвращает их в нормальный вид.

In [11]:
def strings_to_lists(data:pd.DataFrame, columns:list) -> pd.DataFrame:
    '''
    Input:
        data - Dataframe to change
        columns - list of columns to change
    Output:
        Changed DataFrame with values changed from str to list in proper way. 
    '''
    for column in columns:
        data[column] = data[column].str.replace('[\D]+', ' ')
        data[column] = data[column].str.split()
        data[column] = data[column].apply(lambda x: np.array(x).astype('int32'))

    return data

что бы получить top5000 при создании user_item_matrix удаляем  999999 товар(олицетворяющий суммарную чушь купленную не из top 5000) с помощью:

```user_item_matrix.drop([999999], axis=1, inplace=True)```

это сделано в webinar_2.ipynb

In [12]:
result = pd.read_csv('predictions_basic.csv')
result_top5000 = pd.read_csv('predictions_basic_top5000.csv')
result.head(2)

,user_id,actual,random_recommendation,weighted_random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[13944986, 15686741, 12263710, 99835, 6917798]","[9220243, 1011832, 6772676, 833121, 1138192]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[823774, 1062493, 6424437, 1138682, 1112146]","[5592290, 966891, 909744, 5980713, 13512772]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [13]:
result_top5000.head(2)

,user_id,actual,random_recommendation,weighted_random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[981848, 5995494, 1595098, 1130742, 1465377]","[1000486, 12386237, 13987201, 8122541, 1109605]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1127831, 995242, 840361]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 995242, 1029743, 840361, 904360]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[6772706, 9521794, 1457212, 8091195, 1125355]","[856008, 1124432, 10355748, 907131, 898217]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 1098066, 981760, 826249, 883404]","[1082185, 1098066, 981760, 826249, 883404]","[1082185, 1098066, 6534178, 826249, 1127831]"


исправляем все столюцы на списки.

In [14]:
columns = ['actual', 'random_recommendation', 'weighted_random_recommendation', 'popular_recommendation', 'itemitem', 'cosine', 'tfidf' , 'own_purchases']
result = strings_to_lists(result, columns)
result_top5000 = strings_to_lists(result_top5000, columns)

In [15]:
result.head(3)

,user_id,actual,random_recommendation,weighted_random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13944986, 15686741, 12263710, 99835, 6917798]","[9220243, 1011832, 6772676, 833121, 1138192]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[823774, 1062493, 6424437, 1138682, 1112146]","[5592290, 966891, 909744, 5980713, 13512772]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[2005109, 15800994, 1124884, 1058046, 1077377]","[7441191, 1029421, 919294, 950272, 5589845]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 878996]","[999999, 1082185, 1029743, 6534178, 1127831]"


In [16]:
from metrics import precision_at_k, precision

считаем

In [17]:
precisions, precisions_top5000 = [], []
columns.remove('actual')
for algoritm in columns:
    precisions.append(result.apply(lambda row: precision_at_k(row[algoritm], row['actual'], k=5), axis=1).mean())
    precisions_top5000.append(result_top5000.apply(lambda row: precision_at_k(row[algoritm], row['actual'], k=5), axis=1).mean())

pd.DataFrame({'Algoritms': columns, 'Precision@5': precisions, 'Precision@5 top5000': precisions_top5000})

z:\Projects\python\2021.03.20_Recommedation_systems\lesson2_dz\metrics.py:43: RuntimeWarning: invalid value encountered in long_scalars
  precision = flags.sum() / recommended_list.size


,Algoritms,Precision@5,Precision@5 top5000
0,random_recommendation,0.000686,0.000588
1,weighted_random_recommendation,0.000979,0.001763
2,popular_recommendation,0.155240,0.155240
3,itemitem,0.136925,0.155326
4,cosine,0.132909,0.154639
5,tfidf,0.138981,0.158665
6,own_purchases,0.179693,0.219858


В принципе видно, что Weighte_random_recommendation работает средне между random и popular весьма ожидаемо.

Так же видно что результаты сильно лучше когда мы удаляем фиктивный товар 999999, он впринципе не наделен особым смыслом, в нашей задаче, он вспомогательный для того что бы убрать все что не из топ5000. и по сути в user_item_matrix все его значения равны 1, что сильно портит общую картину так как он становится товаром который все купили. Во многих моделях он и будет всем советоваться. Хотя просто его наличие в рекомендациях уже ошибка так как такого товара нету.

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров(done)
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

вторая часть в файле webinar_2.ipynb